# Exploring Loan Data from Prosper
## by Aastha Arora

## Preliminary Wrangling

> Prosper is a peer-to-peer lending marketplace. The [data set](https://www.google.com/url?q=https://s3.amazonaws.com/udacity-hosted-downloads/ud651/prosperLoanData.csv&sa=D&ust=1554486256021000) contains 113,937 loans (last updated 03/11/2014) with 81 variables on each loan, including loan amount, borrower rate (or interest rate), current loan status, borrower income, and many others.

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [2]:
# load the dataset into a pandas dataframe
df = pd.read_csv('prosperLoanData.csv', parse_dates=['ListingCreationDate'])
df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


The variable definitions for all columns in the dataset can be viewed using [link](https://www.google.com/url?q=https://docs.google.com/spreadsheet/ccc?key%3D0AllIqIyvWZdadDd5NTlqZ1pBMHlsUjdrOTZHaVBuSlE%26usp%3Dsharing&sa=D&ust=1554486256024000)

In [3]:
# high-level overview of data shape and composition
print(df.shape)

(113937, 81)


In [4]:
df.dtypes.value_counts()

float64           50
object            16
int64             11
bool               3
datetime64[ns]     1
dtype: int64

**Data Cleaning**

In [5]:
# The dataset contains 81 columns. 
# Filtering selected columns of interest and creating a new dataframe to simplify exploration.

loan_df = df[['ListingCreationDate', 'CreditGrade', 'LoanStatus', 'BorrowerAPR', \
           'BorrowerRate', 'ProsperRating (Alpha)','ProsperScore', 'EmploymentStatus', 'EmploymentStatusDuration', \
           'CreditScoreRangeLower', 'CreditScoreRangeUpper', 'DebtToIncomeRatio', 'IncomeRange', \
           'StatedMonthlyIncome', 'LoanOriginalAmount', 'MonthlyLoanPayment']].copy()

In [6]:
loan_df.shape

(113937, 16)

In [7]:
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 16 columns):
ListingCreationDate         113937 non-null datetime64[ns]
CreditGrade                 28953 non-null object
LoanStatus                  113937 non-null object
BorrowerAPR                 113912 non-null float64
BorrowerRate                113937 non-null float64
ProsperRating (Alpha)       84853 non-null object
ProsperScore                84853 non-null float64
EmploymentStatus            111682 non-null object
EmploymentStatusDuration    106312 non-null float64
CreditScoreRangeLower       113346 non-null float64
CreditScoreRangeUpper       113346 non-null float64
DebtToIncomeRatio           105383 non-null float64
IncomeRange                 113937 non-null object
StatedMonthlyIncome         113937 non-null float64
LoanOriginalAmount          113937 non-null int64
MonthlyLoanPayment          113937 non-null float64
dtypes: datetime64[ns](1), float64(9), int64(1), object(5)
m

**The variable definitions of the dataset highlights that the ProsperScore is applicable for loans originated after July 2009. This seems to be the reason for missing values in this column.**

**CreditGrade**: The Credit rating that was assigned at the time the listing went live. Applicable for listings pre-2009 period and will only be populated for those listings.

**ProsperRating (Alpha)**: The Prosper Rating assigned at the time the listing was created between AA - HR.  Applicable for loans originated after July 2009.

These two columns provide the same data and can be combined to form a single 'ProsperRating' column.

In [8]:
loan_df['CreditGrade'].unique()

array(['C', nan, 'HR', 'AA', 'D', 'B', 'E', 'A', 'NC'], dtype=object)

In [9]:
loan_df['ProsperRating (Alpha)'].unique()

array([nan, 'A', 'D', 'B', 'E', 'C', 'AA', 'HR'], dtype=object)

In [10]:
# Creating a copy of the original loan_df dataset
loan = loan_df.copy()

In [11]:
# Changing the data type of columns to concatenate the columns and creating a new 'ProsperRating' column
loan['ProsperRating'] = loan['CreditGrade'].astype(str).replace('nan','') + \
                        loan['ProsperRating (Alpha)'].astype(str).replace('nan','')

In [12]:
# Test the code
loan.sample(10).loc[:,['CreditGrade', 'ProsperRating (Alpha)', 'ProsperRating']]

,CreditGrade,ProsperRating (Alpha),ProsperRating
72219,NaN,C,C
93017,NaN,C,C
102330,NaN,C,C
29135,NaN,A,A
88810,NaN,B,B
50026,NaN,A,A
7664,NaN,HR,HR
41419,NaN,C,C
74740,NaN,C,C
6121,NaN,D,D


In [13]:
# Dataset does not have any row where both 'CreditGrade' and 'ProsperRating (Alpha)' have non-null values.
loan[(loan['CreditGrade'].notnull()) & (loan['ProsperRating (Alpha)'].notnull())]

,ListingCreationDate,CreditGrade,LoanStatus,BorrowerAPR,BorrowerRate,ProsperRating (Alpha),ProsperScore,EmploymentStatus,EmploymentStatusDuration,CreditScoreRangeLower,CreditScoreRangeUpper,DebtToIncomeRatio,IncomeRange,StatedMonthlyIncome,LoanOriginalAmount,MonthlyLoanPayment,ProsperRating


In [14]:
# Removing the 'CreditGrade' and 'ProsperRating (Alpha)' as these are now redundant
loan.drop(columns = ['CreditGrade','ProsperRating (Alpha)'], inplace=True)

In [15]:
# Test the code
loan.shape

(113937, 15)

In [16]:
loan.ProsperRating.value_counts()

C     23994
B     19970
D     19427
A     17866
E     13084
HR    10443
AA     8881
NC      141
        131
Name: ProsperRating, dtype: int64

In [17]:
loan.loc[(loan.ProsperRating == '') | (loan.ProsperRating == 'NC'), 'ProsperRating'] = np.nan

In [18]:
loan.ProsperRating.value_counts()

C     23994
B     19970
D     19427
A     17866
E     13084
HR    10443
AA     8881
Name: ProsperRating, dtype: int64

In [19]:
# convert ProsperRating and IncomeRating into ordered categorical types
ordinal_var_dict = {'ProsperRating': ['HR', 'E', 'D', 'C', 'B', 'A', 'AA'],
                    'IncomeRange': ['Not displayed', 'Not employed', '$0', '$1-24,999', '$25,000-49,999',\
                    '$50,000-74,999', '$75,000-99,999', '$100,000+']}

for var in ordinal_var_dict:

    ordered_var = pd.api.types.CategoricalDtype(ordered = True,
                                                categories = ordinal_var_dict[var])
    loan[var] = loan[var].astype(ordered_var)

In [20]:
# Test
loan.dtypes

ListingCreationDate         datetime64[ns]
LoanStatus                          object
BorrowerAPR                        float64
BorrowerRate                       float64
ProsperScore                       float64
EmploymentStatus                    object
EmploymentStatusDuration           float64
CreditScoreRangeLower              float64
CreditScoreRangeUpper              float64
DebtToIncomeRatio                  float64
IncomeRange                       category
StatedMonthlyIncome                float64
LoanOriginalAmount                   int64
MonthlyLoanPayment                 float64
ProsperRating                     category
dtype: object

In [21]:
loan.sample(5)

,ListingCreationDate,LoanStatus,BorrowerAPR,BorrowerRate,ProsperScore,EmploymentStatus,EmploymentStatusDuration,CreditScoreRangeLower,CreditScoreRangeUpper,DebtToIncomeRatio,IncomeRange,StatedMonthlyIncome,LoanOriginalAmount,MonthlyLoanPayment,ProsperRating
17783,2011-08-01 09:28:00.190,Current,0.20200,0.1799,8.0,Employed,58.0,700.0,719.0,0.21,"$25,000-49,999",3750.000000,2000,72.29,B
75333,2011-05-31 11:46:14.763,Completed,0.27467,0.2399,5.0,Employed,136.0,680.0,699.0,0.45,"$50,000-74,999",5962.000000,5500,215.75,D
96914,2012-03-14 06:17:36.597,Completed,0.12782,0.0999,9.0,Employed,97.0,760.0,779.0,0.19,"$100,000+",8789.416667,10000,322.62,A
100956,2013-04-16 13:08:13.517,Completed,0.25781,0.2199,4.0,Employed,79.0,680.0,699.0,0.10,"$100,000+",8500.000000,15000,572.78,D
32911,2008-04-24 09:18:57.800,Chargedoff,0.11853,0.0975,NaN,Self-employed,51.0,560.0,579.0,NaN,$0,0.000000,5000,160.75,E


In [22]:
# Summary Statistics
loan.describe()

,BorrowerAPR,BorrowerRate,ProsperScore,EmploymentStatusDuration,CreditScoreRangeLower,CreditScoreRangeUpper,DebtToIncomeRatio,StatedMonthlyIncome,LoanOriginalAmount,MonthlyLoanPayment
count,113912.000000,113937.000000,84853.000000,106312.000000,113346.000000,113346.000000,105383.000000,1.139370e+05,113937.00000,113937.000000
mean,0.218828,0.192764,5.950067,96.071582,685.567731,704.567731,0.275947,5.608026e+03,8337.01385,272.475783
std,0.080364,0.074818,2.376501,94.480605,66.458275,66.458275,0.551759,7.478497e+03,6245.80058,192.697812
min,0.006530,0.000000,1.000000,0.000000,0.000000,19.000000,0.000000,0.000000e+00,1000.00000,0.000000
25%,0.156290,0.134000,4.000000,26.000000,660.000000,679.000000,0.140000,3.200333e+03,4000.00000,131.620000
50%,0.209760,0.184000,6.000000,67.000000,680.000000,699.000000,0.220000,4.666667e+03,6500.00000,217.740000
75%,0.283810,0.250000,8.000000,137.000000,720.000000,739.000000,0.320000,6.825000e+03,12000.00000,371.580000
max,0.512290,0.497500,11.000000,755.000000,880.000000,899.000000,10.010000,1.750003e+06,35000.00000,2251.510000
